In [16]:
import pandas as pd
import pyodbc
from fuzzywuzzy import fuzz
import datetime as dt

In [18]:
import pyodbc
username = 'vigrose'
password = 'Hufflepuff10946'
s = "DSN=PRDDM; UID={}; PWD={}".format(username, password)
AMADM = pyodbc.connect(s)

In [3]:
s = "DSN=PRDDM; UID={}; PWD={}".format(username, password)
AMADM = pyodbc.connect(s)

In [13]:
ppd = pd.read_csv('../../Data/Scorecard/ppd_data_20200530.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,49,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
today_phys_query= \
    """ 
    SELECT DISTINCT 
    D.FULL_DT,
    H.MED_EDU_NBR AS ME,
    H.PARTY_ID,
    O.ORDER_NBR,
    O.ORDER_PRODUCT_ID, 
    O.ORDER_PHYSICIAN_HIST_KEY,
    O.CUSTOMER_KEY,
    O.CUSTOMER_NBR
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND
    D.MONTH_NBR = 5
    AND
    D.YR = 2020
    """

In [11]:
query = \
    """ 
    SELECT DISTINCT 
    P.PRODUCT_ID,
    P.PRODUCT_DESC
    FROM
    AMADM.DIM_PRODUCT P
    """

In [26]:
query = \
    """
    SELECT DISTINCT
    C.CUSTOMER_KEY, 
    C.CUSTOMER_NBR,
    C.CUSTOMER_ISELL_LOGIN,
    C.CUSTOMER_NAME,
    C.CUSTOMER_TYPE_DESC,
    C.CUSTOMER_TYPE,
    C.CUSTOMER_CATEGORY_DESC
    FROM 
    AMADM.dim_customer c  
    """

In [27]:
PHYS = pd.read_sql(con=AMADM, sql=query)
PHYS

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,1,000000105762,35HP10545,Nassau County Medical Center,Eprofile Webstore Cusomer,Eprofile,Hospital
1,2,000000545697,None,Abdul Haji,Non-Physician Individual,Nonphy,None
2,3,000000552782,None,Mary Tan,Non-Physician Individual,Nonphy,None
3,4,000000647381,None,Osman Kozak,Physician (On Aims),Phys,None
4,5,000000648448,None,Rana Kayal,Non-Physician Individual,Nonphy,None
...,...,...,...,...,...,...,...
145023,145163,002416110649,None,Jennifer Menon,Physician (On Aims),Phys,None
145024,145164,005606191971,None,Jeffrey Farvour,Us Physician Resident/Fellow,Resident,None
145025,145165,001102130720,None,Maggie Attia,Physician (On Aims),Phys,None
145026,145166,013201120033,None,Matias Amaral,Physician (On Aims),Phys,None


In [28]:
org = pd.read_excel("../../Data/Credentialling/Org_Addresses.xlsx")

In [36]:
PHYS.CUSTOMER_NBR.astype('int64')

0              105762
1              545697
2              552782
3              647381
4              648448
             ...     
145023     2416110649
145024     5606191971
145025     1102130720
145026    13201120033
145027    91504000104
Name: CUSTOMER_NBR, Length: 145028, dtype: int64

In [31]:
org['Customer Number']

0             13323
1             17736
2             18125
3             18760
4            486394
            ...    
11407       4007512
11408       4009224
11409       4013302
11410    2501871519
11411    3515911307
Name: Customer Number, Length: 11412, dtype: int64

In [46]:
cred = pd.merge(PHYS, ppd, on='ME')[['FULL_DT','POLO_STATE','POLO_CITY','CUSTOMER_KEY','ORDER_PRODUCT_ID']]

In [49]:
cred[cred['POLO_STATE']=='PA']

,FULL_DT,POLO_STATE,POLO_CITY,CUSTOMER_KEY,ORDER_PRODUCT_ID
64,2020-05-22,PA,BALA CYNWYD,54868,4915513
240,2020-05-18,PA,WEST CHESTER,56357,4915513
303,2020-05-29,PA,READING,69253,4915514
319,2020-05-08,PA,NORRISTOWN,54998,4915514
385,2020-05-22,PA,JOHNSTOWN,54868,4915513
...,...,...,...,...,...
47148,2020-05-29,PA,READING,69253,4915514
47201,2020-05-27,PA,PITTSBURGH,136716,4915491
47253,2020-05-29,PA,PITTSBURGH,53389,4915503
47254,2020-05-29,PA,PITTSBURGH,53389,4922510


In [5]:
prod_query= \
    """ 
    SELECT DISTINCT
    P.PRODUCT_ID,
    P.PRODUCT_DESC,
    P.PRODUCT_CAT_CD,
    P.PRODUCT_CAT_DESC
    FROM 
    AMADM.DIM_PRODUCT P
    """
products = pd.read_sql(con=AMADM, sql=prod_query)
products.head()

,PRODUCT_ID,PRODUCT_DESC,PRODUCT_CAT_CD,PRODUCT_CAT_DESC
0,4899372,?,MIS,AMA PRESS
1,4899373,None,MIS,AMA PRESS
2,4899374,?,MIS,AMA PRESS
3,4899375,?,MIS,AMA PRESS
4,4899376,None,MIS,AMA PRESS


In [113]:
pd.merge(products, MAY_PHYS, left_on='PRODUCT_ID',right_on='ORDER_PRODUCT_ID')[['PRODUCT_DESC','PRODUCT_ID']].drop_duplicates().to_csv('../../Data/Credentialling/product_key.csv', index=False)

In [79]:
MAY_PHYS.groupby('ORDER_PRODUCT_ID').count()

,FULL_DT,ME,ORDER_NBR,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY
ORDER_PRODUCT_ID,,,,,
4915491,1009,1009,1009,1009,1009
4915494,765,765,765,765,765
4915501,33,33,33,33,33
4915502,34,34,34,34,34
4915503,669,669,669,669,669
4915508,51,51,51,51,51
4915509,48,48,48,48,48
4915512,904,904,904,904,904
4915513,30095,30095,30095,30095,30095


In [4]:
hosp_cust_query = \
    """
    SELECT DISTINCT
    C.CUSTOMER_KEY, 
    C.CUSTOMER_NBR,
    C.CUSTOMER_ISELL_LOGIN,
    C.CUSTOMER_NAME,
    C.CUSTOMER_TYPE_DESC,
    C.CUSTOMER_TYPE,
    C.CUSTOMER_CATEGORY_DESC
    FROM 
    AMADM.dim_customer c    
    """

In [5]:
HOSP_CUST = pd.read_sql(con=AMADM, sql=hosp_cust_query)
HOSP_CUST.CUSTOMER_KEY = HOSP_CUST.CUSTOMER_KEY.astype(str)
HOSP_CUST = HOSP_CUST.fillna('None')
HOSP_CUST.head()

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,1,000000105762,35HP10545,Nassau County Medical Center,Eprofile Webstore Cusomer,Eprofile,Hospital
1,2,000000545697,None,Abdul Haji,Non-Physician Individual,Nonphy,None
2,3,000000552782,None,Mary Tan,Non-Physician Individual,Nonphy,None
3,4,000000647381,None,Osman Kozak,Physician (On Aims),Phys,None
4,5,000000648448,None,Rana Kayal,Non-Physician Individual,Nonphy,None


In [8]:
class_cust = HOSP_CUST[(HOSP_CUST['CUSTOMER_CATEGORY_DESC']!='None')&(HOSP_CUST['CUSTOMER_NAME']!='None')]

In [18]:
CUSTOMER_ = pd.merge(class_cust, cred, on='CUSTOMER_KEY')
CUSTOMER_ = CUSTOMER_.fillna('None')

In [21]:
dict_list = []
customer_dict = {}
for row in CUSTOMER_.itertuples():
    NEW = False
    if row.CUSTOMER_KEY not in customer_dict.keys():
        NEW = True
        customer_dict[row.CUSTOMER_KEY]=[row.POLO_STATE]
    elif row.POLO_STATE not in customer_dict[row.CUSTOMER_KEY] and row.POLO_STATE!='None':
        NEW = True
        customer_dict[row.CUSTOMER_KEY].append(row.POLO_STATE)
    if NEW == True:
        new_dict = {
        'CUSTOMER_NAME': row.CUSTOMER_NAME,
        'CUSTOMER_KEY': row.CUSTOMER_KEY,
        'STATE': row.POLO_STATE,
        'CATEGORY': row.CUSTOMER_CATEGORY_DESC,
        'FULLT_DT':row.FULL_DT
    }
        dict_list.append(new_dict)

In [26]:
customer_states = pd.DataFrame(dict_list)

In [27]:
customer_states.to_csv('../../Data/Credentialling/Customer_States.csv')

In [47]:
customer_orders = pd.merge(MAY_PHYS, class_cust, on='CUSTOMER_KEY')

In [77]:
may_customers.groupby('CUSTOMER_CATEGORY_DESC').count()[['CUSTOMER_KEY']]

,CUSTOMER_KEY
CUSTOMER_CATEGORY_DESC,
Ambulatory Care,3728
Consulting,150
County Medical Society,37
Credentialling Board,204
Credit Verification Org,2963
Government,86
Group,1842
Health Related,4040
Hospital,22278


In [38]:
may_customers = pd.merge(class_cust, may_cred, on='CUSTOMER_KEY')

In [44]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums

In [45]:
ppd['ME']=fix_me(ppd['ME'])

In [17]:
may_cred.groupby('PE_CD').count()[['FULL_DT']]

,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,CUSTOMER_KEY
PE_CD,,,,,,
11,3414,3414,3414,2731,3414,3414
13,831,831,831,664,831,831
21,254,254,254,199,254,254
22,126,126,126,106,126,126
30,15082,15082,15082,12509,15082,15082
35,28,28,28,24,28,28
40,367,367,367,284,367,367
50,13093,13093,13093,9476,13092,13093
63,2692,2692,2692,2177,2692,2692


In [37]:
class_cust['CUSTOMER_KEY'] = class_cust['CUSTOMER_KEY'].astype(str)

In [39]:
may_customers = may_customers.fillna('None')
unclass = may_customers[(may_customers['CUSTOMER_CATEGORY_DESC']!='None')&(may_customers['PE_CD']==110)]

In [41]:
hosp_may[['CUSTOMER_NAME','CUSTOMER_NBR','CUSTOMER_KEY','FULL_DT','ME','FIRST_NAME','MIDDLE_NAME','LAST_NAME']].to_csv('../../Data/Credentialling/Credentialing_Order_Affiliation_Product_2020_05.csv')

In [75]:
MAY_STUFF = hosp_may[['CUSTOMER_NAME','CUSTOMER_NBR','CUSTOMER_KEY','FULL_DT','ME','FIRST_NAME','MIDDLE_NAME','LAST_NAME']]

In [43]:
HOSP_CUST.to_csv('../../Data/Credentialling/Hospital_Customers.csv')

In [52]:
# ppd = ppd.fillna('None')
CHECK_12 = ppd[(ppd['POLO_MAILING_LINE_1']!='None')&(ppd['PE_CD']!=110)][['POLO_MAILING_LINE_1','PE_CD','POLO_STATE']]

In [74]:
CHECK_12[CHECK_12.POLO_MAILING_LINE_1 == 'MONROE CLINIC']

,POLO_MAILING_LINE_1,PE_CD,POLO_STATE
89085,MONROE CLINIC,30,WI
203003,MONROE CLINIC,50,WI
210398,MONROE CLINIC,30,WI
212646,MONROE CLINIC,63,WI
221645,MONROE CLINIC,30,WI
235713,MONROE CLINIC,30,WI
396970,MONROE CLINIC,11,WI
398697,MONROE CLINIC,63,WI
486915,MONROE CLINIC,30,WA
659068,MONROE CLINIC,63,WI


In [65]:
customer_list = [x.upper() for x in list(unclass['CUSTOMER_NAME'].unique())]

In [69]:
CHECK_12[CHECK_12.POLO_MAILING_LINE_1.isin(customer_list)].drop_duplicates('POLO_MAILING_LINE_1')

,POLO_MAILING_LINE_1,PE_CD,POLO_STATE
162,NORTHSHORE UNIVERSITY HEALTH SYSTEM,30,IL
485,UNIVERSITY OF MARYLAND MEDICAL CENTER,50,MD
599,MEMPHIS MENTAL HEALTH INSTITUTE,30,TN
809,ATLANTA VAMC,85,GA
847,KAISER PERMANENTE,30,HI
...,...,...,...
1266483,TWIN CITIES COMMUNITY HOSPITAL,63,CA
1267786,SUBURBAN COMMUNITY HOSPITAL,30,PA
1274646,MEMORIAL HOSPITAL OF SOUTH BEND,63,IN
1274961,HILLSDALE COMMUNITY HEALTH CENTER,86,MI


In [76]:
for name in customer_list:
    if 'VETERAN' in name:
        print(name)
    elif 'VAMC' in name:
        print(name)
    elif ' VA ' in name:
        print(name)

PHILADELPHIA VA MEDICAL CENTER
JAMES E. VAN ZANDT VA MEDICAL CENTER
EDWARD HINES JR. VA  HOSPITAL
VA GULF COAST VETERANS HLTH CARE SYS
CENTRAL ALABAMA VETERANS HEALTHCARE SYSTEM
FAYETTEVILLE VETERANS ADMINISTRATION MEDICAL CENTER
ATLANTA VAMC
ORLANDO VA MEDICAL CENTER
JESSE BROWN VA MEDICAL CENTER
PITTSBURGH VA HEALTHCARE SYSTEM
HARRY S TRUMAN MEMORIAL VETERANS' HOSPITAL
WM. S. MIDDLETON MEMORIAL VA HOSPITAL
PORTLAND VA MEDICAL CENTER
SOUTH TEXAS VETERANS HEALTH CARE SYSTEM
CARL T. HAYDEN VA MED CTR (05D)
CLEMENT J. ZABLOCKI VA MEDICAL CENTER
VETERANS AFFAIRS MEDICAL CENTER
ROUDEBUSH VA MEDICAL CENTER
MINNEAPOLIS VA MEDICAL CENTER
LOUIS STOKES CLEVELAND VA MEDICAL CENTER
SAN FRANCISCO VA HEALTH CARE SYSTEM
CENTRAL TEXAS VETERANS HEALTHCARE SYSTEM
BOISE VETERANS AFFAIRS MEDICAL CENTER
BECKLEY VA MEDICAL CENTER


In [70]:
unclass.groupby('CUSTOMER_CATEGORY_DESC').count()

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,PE_CD
CUSTOMER_CATEGORY_DESC,,,,,,,,,,,,
Ambulatory Care,830,830,830,830,830,830,830,830,830,830,830,830
Consulting,40,40,40,40,40,40,40,40,40,40,40,40
County Medical Society,16,16,16,16,16,16,16,16,16,16,16,16
Credentialling Board,50,50,50,50,50,50,50,50,50,50,50,50
Credit Verification Org,718,718,718,718,718,718,718,718,718,718,718,718
Government,20,20,20,20,20,20,20,20,20,20,20,20
Group,431,431,431,431,431,431,431,431,431,431,431,431
Health Related,947,947,947,947,947,947,947,947,947,947,947,947
Hospital,5219,5219,5219,5219,5219,5219,5219,5219,5219,5219,5219,5219


In [78]:
unclass[unclass.CUSTOMER_CATEGORY_DESC=='Government']

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,PE_CD
1066,52312,000002100445,41HP80242,James E. Van Zandt Va Medical Center,Eprofile Webstore Customer,Eprofile,Government,2020-05-06,01803151161,PETER,WILLIAM,WEHR,110
3738,53333,000002196473,None,Central Alabama Veterans Healthcare System,Eprofile Webstore Customer,Eprofile,Government,2020-05-05,42201137874,NISHANT,CHRISTOPHER,KHATRI,110
13392,55316,000002189125,03GO51969,Phoenix Area Ihs,Eprofile Webstore Customer,Eprofile,Government,2020-05-13,05107020367,ROBERT,MARTIN,REUTER,110
13394,55316,000002189125,03GO51969,Phoenix Area Ihs,Eprofile Webstore Customer,Eprofile,Government,2020-05-12,03547090127,AIMEE,None,LEE,110
13395,55316,000002189125,03GO51969,Phoenix Area Ihs,Eprofile Webstore Customer,Eprofile,Government,2020-05-12,20901930067,MYO,None,MYINT,110
13396,55316,000002189125,03GO51969,Phoenix Area Ihs,Eprofile Webstore Customer,Eprofile,Government,2020-05-12,02312110868,JONATHAN,TONY,LAU,110
13398,55316,000002189125,03GO51969,Phoenix Area Ihs,Eprofile Webstore Customer,Eprofile,Government,2020-05-12,91502960461,GEHAN,RIZK LABIB,BOTRUS,110
13399,55316,000002189125,03GO51969,Phoenix Area Ihs,Eprofile Webstore Customer,Eprofile,Government,2020-05-11,42201121315,WAQAS,AHMAD,QAMAR,110
13400,55316,000002189125,03GO51969,Phoenix Area Ihs,Eprofile Webstore Customer,Eprofile,Government,2020-05-29,14303150049,CHARLES,JOSEPH RICHARD,GAULIN,110
13402,55316,000002189125,03GO51969,Phoenix Area Ihs,Eprofile Webstore Customer,Eprofile,Government,2020-05-12,03305021112,DOUGLAS,CHINLU,CHANG,110


In [45]:
npi_hosp = pd.read_csv('../../Data/NPI/NPI_Hospitals_20-01-12.csv')
amc_hosp = pd.read_csv('../../Data/NPI/AMC_Hospital_Dataset_20-02-18.csv')

In [58]:
# HOSP_CUST['CUSTOMER_NAME'] = HOSP_CUST['CUSTOMER_NAME'].str.upper()


on_name = pd.merge(HOSP_CUST, npi_hosp, left_on = 'CUSTOMER_NAME', right_on = 'NAME')
on_other_name = pd.merge(HOSP_CUST, npi_hosp, left_on = 'CUSTOMER_NAME', right_on = 'OTHER_NAME')


In [73]:
HOSP_CUST[HOSP_CUST.CUSTOMER_NBR.isin(list(npi_match.CUSTOMER_NBR))==False].to_csv('../../Data/Credentialling/UNMATCHED.csv', index=False)

In [66]:
npi_match = all_match.sort_values('LAST_UPDATE_DATE').drop_duplicates('CUSTOMER_NBR', keep ='last')

In [63]:
# all_match = pd.concat([on_name, on_other_name])
all_match['LAST_UPDATE_DATE'] = pd.to_datetime(all_match['LAST_UPDATE_DATE'])

In [52]:
HOSP_CUST = HOSP_CUST.drop_duplicates('CUSTOMER_NBR')

In [51]:
hosp_may[['CUSTOMER_NAME','CUSTOMER_NBR','CUSTOMER_KEY','FULL_DT','ME','FIRST_NAME','MIDDLE_NAME','LAST_NAME']].drop_duplicates()

,CUSTOMER_NAME,CUSTOMER_NBR,CUSTOMER_KEY,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME
0,River Region Health System,000002099306,51732,2020-05-11,03509131154,JONATHAN,HANS,CHEN
1,River Region Health System,000002099306,51732,2020-05-13,69018050030,OLUWAROTIMI,ADEYEMI,ADEPOJU
2,River Region Health System,000002099306,51732,2020-05-27,02106150535,RYAN,WILLIAM,BROWN
3,River Region Health System,000002099306,51732,2020-05-18,49570110097,THULASI,RAM,GUDI
4,River Region Health System,000002099306,51732,2020-05-07,02407871262,JOAN,HAZEL,SUTCLIFFE
...,...,...,...,...,...,...,...,...
22118,Kingman Regional Medical Center,000002080332,136726,2020-05-30,02312980374,WILLIAM,NaN,HALLIER
22119,George Washington Medical Faculty Associates,000002080801,136743,2020-05-29,02101880187,KIM,MARIA,CALLWOOD
22121,George Washington Medical Faculty Associates,000002080801,136743,2020-05-28,02512881192,YVONNE,RENE,REID
22123,George Washington Medical Faculty Associates,000002080801,136743,2020-05-28,49607720150,VIMLA,NaN,BHOOSHAN


In [101]:
check_12 = pd.merge(npi_match, MAY_STUFF, on = 'CUSTOMER_NBR')

In [104]:
check_12.to_csv('../../Data/Credentialling/Credentialing_Order_Affiliation_Product.csv', index=False)

In [79]:
len(MAY_STUFF)

22127

In [98]:
states = ppd[['ME','POLO_STATE']]

In [82]:
len(MAY_PHYS)

48932

In [84]:
len(hosp_may)

22127

In [86]:
hosp_may.groupby('CUSTOMER_NAME').count()

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME
CUSTOMER_NAME,,,,,,,,,,,
Abbeville General Hospital,13,13,13,13,13,13,13,13,13,12,13
Abington Memorial Hospital,28,28,28,28,28,28,28,28,28,21,28
Abraham Lincoln Memorial Hospital,2,2,2,2,2,2,2,2,2,1,2
Accel Rehabilitation Hospital Of Plano,3,3,0,3,3,3,3,3,3,1,3
Acmh Hospital,4,4,4,4,4,4,4,4,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...
Yale New Haven Hospital,2,2,2,2,2,2,2,2,2,1,2
Yavapai Regional Medical Center,2,2,2,2,2,2,2,2,2,1,2
Yuma Regional Medical Center,9,9,9,9,9,9,9,9,9,5,9


In [89]:
UNQ_HOSP = hosp_may.drop_duplicates('CUSTOMER_NAME')

In [95]:
XX = UNQ_HOSP[UNQ_HOSP.CUSTOMER_NBR.isin(list(npi_match.CUSTOMER_NBR))==False]

In [100]:
pd.merge(XX, states, on='ME').to_csv('../../Data/Credentialling/may_unmatched.csv', index=False)

In [105]:
len(HOSP_CUST)

4133

In [106]:
len(npi_match)

2173

In [107]:
ppd.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [80]:
dt.datetime.today()

datetime.datetime(2020, 6, 18, 14, 45, 21, 224035)

In [83]:
import dateutil.relativedelta
now = dt.datetime.now()
print (now)
print ((now + dateutil.relativedelta.relativedelta(months=-1)).month)

2020-06-18 14:48:54.982451
5


In [18]:
wslive = pd.read_csv('../../Data/Humach/wslive_with_results_all_20190719.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,5,10,12,14,16,18,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
wslive.dtypes

PHYSICIAN_ME_NUMBER                float64
PHYSICIAN_FIRST_NAME                object
PHYSICIAN_MIDDLE_NAME               object
PHYSICIAN_LAST_NAME                 object
SUFFIX                              object
DEGREE                              object
OFFICE_ADDRESS_LINE_1               object
OFFICE_ADDRESS_LINE_2               object
OFFICE_ADDRESS_CITY                 object
OFFICE_ADDRESS_STATE                object
OFFICE_ADDRESS_ZIP                  object
OFFICE_ADDRESS_VERIFIED_UPDATED    float64
OFFICE_TELEPHONE                    object
OFFICE_PHONE_VERIFIED_UPDATED      float64
OFFICE_FAX                          object
OFFICE_FAX_VERIFIED_UPDATED        float64
EMAIL_ADDRESS                       object
EMAIL_ADDRESS_VERIFIED_UPDATED     float64
PRESENT_EMPLOYMENT_CODE             object
PRESENT_EMPLOYMENT_UPDATED         float64
COMMENTS                            object
Source                              object
WSLIVE_SOURCE                       object
WSLIVE_FILE

In [22]:
wslive.groupby(['pe_status']).count()

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED,ws_month,ws_year,new_metric,addr_status,phone_status,fax_status,spec_status
pe_status,,,,,,,,,,,,,,,,,,,,,
Confirmed,499237,499237,429231,499236,19738,0,297265,499238,499220,499220,...,0,409349,409349,499240,499240,499240,499240,499240,499240,499240
Inconclusive,41235,41234,34112,41233,1315,822,20954,39549,39611,39609,...,3525,29427,29514,41235,41235,41235,41235,41235,41235,41235
Known Bad,275094,275088,229419,275086,8972,9465,128446,258738,258959,258950,...,23487,211077,212885,275094,275094,275094,275094,275094,275094,275094
No Contact,953313,953304,795449,953307,31654,59439,511810,922232,922693,922671,...,43348,120713,121600,953336,953336,953336,953336,953336,953336,953336
Updated,305909,305906,257470,305905,11025,31386,171439,305910,305900,305900,...,100511,117277,120552,306007,306007,306007,306007,306007,306007,306007


In [24]:
wslive.sort_values('ws_year')

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,SPECIALTY,SPECIALTY_UPDATED,ws_month,ws_year,new_metric,addr_status,pe_status,phone_status,fax_status,spec_status
870210,3.601851e+09,MARK,HENRY,KNELSON,NaN,NaN,STE 200,1121 SITUS CT,RALEIGH,NC,...,NaN,NaN,8,2015,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Updated
384631,1.902831e+09,DAVID,LEROY,LEE,NaN,NaN,STE 208,20375 W 151ST ST,OLATHE,KS,...,NaN,NaN,6,2015,No Contact,No Contact,No Contact,No Contact,No Contact,No Contact
1509190,4.956206e+10,NANDAKISHORE,NaN,AKULA,NaN,NaN,CO INTERNAL MEDICINE DEPT,2139 AUBURN AVE,CINCINNATI,OH,...,NaN,NaN,12,2015,Known Bad,Known Bad,Known Bad,Known Bad,Known Bad,Known Bad
384627,1.902831e+09,GORDON,NaN,LANG,NaN,NaN,NaN,333 N 14TH ST,WA KEENEY,KS,...,NaN,NaN,5,2015,Updated,Confirmed,Updated,Confirmed,Confirmed,Updated
384624,1.902831e+09,LAURA,MARIE,KENNY,NaN,NaN,STE 200,7311 W 132ND ST,OVERLAND PARK,KS,...,NaN,NaN,6,2015,No Contact,No Contact,No Contact,No Contact,No Contact,No Contact
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1931204,3.608951e+09,MARTIN,LUTHER,BLUE,NaN,NaN,NaN,134 PROFESSIONAL PARK DR STE 200,MOORESVILLE,NC,...,NaN,NaN,3,2019,No Contact,No Contact,No Contact,No Contact,No Contact,No Contact
1931205,1.175891e+09,DAVID,ALAN,CROLAND,NaN,NaN,NaN,16147 LANCASTER HWY STE 120,CHARLOTTE,NC,...,NaN,NaN,3,2019,No Contact,No Contact,No Contact,No Contact,No Contact,No Contact
1931206,4.178051e+09,DEREK,M,REED,NaN,NaN,NaN,159 E DALLAS RD,STANLEY,NC,...,NaN,NaN,3,2019,No Contact,No Contact,No Contact,No Contact,No Contact,No Contact
1931182,7.481986e+10,MILAGROS,DIAZ,TEVES,NaN,NaN,NaN,205 N BELLE MEAD AVE,EAST SETAUKET,NY,...,NaN,NaN,3,2019,No Contact,No Contact,No Contact,No Contact,No Contact,No Contact


In [27]:
wslive = wslive.fillna('None')
confirmed = wslive[(wslive.addr_status.isin(['Confirmed', 'Updated']))&(wslive.pe_status.isin(['Confirmed', 'Updated']))]
confirmed = confirmed.sort_values('ws_year')
confirmed = confirmed.drop_duplicates(['OFFICE_ADDRESS_LINE_1','OFFICE_ADDRESS_LINE_2'], keep='last')

In [63]:
reference = confirmed[confirmed.OFFICE_ADDRESS_LINE_1!='None'][['OFFICE_ADDRESS_LINE_1','OFFICE_ADDRESS_LINE_2','PRESENT_EMPLOYMENT_CODE']]

In [40]:
unclass['CUSTOMER_NAME']

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,PE_CD
11,51732,000002099306,27HP50130,River Region Health System,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-21,02312140112,MICHAEL,JOHN,BERGE,110
17,51741,000002435304,None,Oro Valley Hospital,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-05,01401110620,VAROON,RATHAN,THAVAPALAN,110
21,51744,000002320655,23HP90001,University Of Maryland Capital Region Health,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-12,79701050024,ABDULLAH,HASAN A,ALZAHRANI,110
33,51752,000002083379,05HP06014,Childrens Hospital Central California,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-21,05107121147,VIDHI,PANKAJ,SHAH,110
36,51759,000002083493,18HP50031,Unitypoint St. Luke'S Hospital - Cedar Rapids,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-19,03547100017,TAKINTOPE,None,AKINBIYI,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47051,136653,000002083374,39HR09179,Cherokee Nation,Eprofile Webstore Customer,Eprofile,Health Related,2020-05-27,00401041125,JIM,W,STALLCUP,110
47078,136665,000003890267,None,Life Coast Community Health Center,Eprofile Webstore Customer,Eprofile,Health Related,2020-05-26,66102160461,JAMES,BRADLEY,HYATT,110
47079,136665,000003890267,None,Life Coast Community Health Center,Eprofile Webstore Customer,Eprofile,Health Related,2020-05-26,02105101204,BABI,REDDY,CHANDAMURI,110
47116,136690,000002086575,51AM73966,Genetics & Ivf Institute,Eprofile Webstore Customer,Eprofile,Ambulatory Care,2020-05-26,01001040240,ABBAA,None,SARHAN,110


In [41]:
customer_list = [x.upper() for x in list(unclass['CUSTOMER_NAME'].unique())]

In [65]:
unclass['CUSTOMER_NAME'] = [x.upper() for x in list(unclass['CUSTOMER_NAME'])]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [73]:
unclass.drop_duplicates(['ME','CUSTOMER_NAME'])

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,PE_CD
11,51732,000002099306,27HP50130,RIVER REGION HEALTH SYSTEM,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-21,02312140112,MICHAEL,JOHN,BERGE,110
17,51741,000002435304,None,ORO VALLEY HOSPITAL,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-05,01401110620,VAROON,RATHAN,THAVAPALAN,110
21,51744,000002320655,23HP90001,UNIVERSITY OF MARYLAND CAPITAL REGION HEALTH,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-12,79701050024,ABDULLAH,HASAN A,ALZAHRANI,110
33,51752,000002083379,05HP06014,CHILDRENS HOSPITAL CENTRAL CALIFORNIA,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-21,05107121147,VIDHI,PANKAJ,SHAH,110
36,51759,000002083493,18HP50031,UNITYPOINT ST. LUKE'S HOSPITAL - CEDAR RAPIDS,Eprofile Webstore Customer,Eprofile,Hospital,2020-05-19,03547100017,TAKINTOPE,None,AKINBIYI,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47051,136653,000002083374,39HR09179,CHEROKEE NATION,Eprofile Webstore Customer,Eprofile,Health Related,2020-05-27,00401041125,JIM,W,STALLCUP,110
47078,136665,000003890267,None,LIFE COAST COMMUNITY HEALTH CENTER,Eprofile Webstore Customer,Eprofile,Health Related,2020-05-26,66102160461,JAMES,BRADLEY,HYATT,110
47079,136665,000003890267,None,LIFE COAST COMMUNITY HEALTH CENTER,Eprofile Webstore Customer,Eprofile,Health Related,2020-05-26,02105101204,BABI,REDDY,CHANDAMURI,110
47116,136690,000002086575,51AM73966,GENETICS & IVF INSTITUTE,Eprofile Webstore Customer,Eprofile,Ambulatory Care,2020-05-26,01001040240,ABBAA,None,SARHAN,110


In [76]:
pd.merge(unclass, reference, left_on='CUSTOMER_NAME', right_on='OFFICE_ADDRESS_LINE_1', how='left').drop_duplicates(['ME','CUSTOMER_NAME']).to_csv('../../Data/Credentialling/PE_UPDATE_MAY.csv')

In [64]:
xx = reference[reference.OFFICE_ADDRESS_LINE_1.isin(customer_list)].drop_duplicates('OFFICE_ADDRESS_LINE_1')

In [49]:
reference = reference.drop_duplicates('OFFICE_ADDRESS_LINE_1')
reference_list = list(reference['OFFICE_ADDRESS_LINE_1'])

In [55]:
dict_list = []
for customer in customer_list:
    for reference in reference_list:
        score = fuzz.ratio(customer, reference)
        if score >90:
            new_dict = {
                'CUSTOMER_NAME': customer,
                'OFFICE_ADDRESS_LINE_1': reference,
                'SCORE': score
            }
            dict_list.append(new_dict)
            print(f'{customer} is {reference}')
            break

RIVER REGION HEALTH SYSTEM is RIVER REGION HEALTH SYSTEM
ORO VALLEY HOSPITAL is ORO VALLEY HOSPITAL
CHILDRENS HOSPITAL CENTRAL CALIFORNIA is CHILDREN'S HOSPITAL CENTRAL CALIFORNIA
MONROE CLINIC is MONROE CLINIC
NEW LONDON HOSPITAL is NEW LONDON HOSPITAL
OZARKS MEDICAL CENTER is OZARK MEDICAL CENTER
LOGANSPORT MEMORIAL HOSPITAL is LOGANSPORT MEMORIAL HOSPITAL
ARMED FORCES RETIREMENT HOME is ARMED FORCES RETIREMENT HOME
OREGON STATE HOSPITAL is OREGON STATE HOSPITAL
MAT-SU REGIONAL MEDICAL CENTER is MATSU REGIONAL MEDICAL CENTER
BSA HEALTH SYSTEMS is BSA HEALTH SYSTEM
UNITED HEALTH SERVICES is UNITED HEALTH SERVICES INC
VISTA MEDICAL CENTER is VA MEDICAL CENTER
PHILADELPHIA VA MEDICAL CENTER is PHILADELPHIA VA MEDICAL CENTER
UNIVERSITY OF UTAH HEALTH CARE is UNIVERSITY OF UTAH HEALTH CARE
ABINGTON MEMORIAL HOSPITAL is ABINGTON MEMORIAL HOSPITAL
SOUTH BEND CLINIC is SOUTHEND CLINIC
MOLINA HEALTHCARE is MOLINA HLTHCARE
LAKE NORMAN REGIONAL MEDICAL CENTER is LAKE NORMAN REGIONAL MEDICAL CEN

KeyboardInterrupt: 

['RIVER REGION HEALTH SYSTEM',
 'ORO VALLEY HOSPITAL',
 'UNIVERSITY OF MARYLAND CAPITAL REGION HEALTH',
 'CHILDRENS HOSPITAL CENTRAL CALIFORNIA',
 "UNITYPOINT ST. LUKE'S HOSPITAL - CEDAR RAPIDS",
 'MONROE CLINIC',
 'HEALTHCARE AUTHORITY FOR MEDICAL WEST',
 'NORTH TAMPA BEHAVIORAL HEALTH',
 'MISSISSIPPI VALLEY SURGERY CENTER',
 'PROVIDENCE HEALTH AND SERVICES CALIFORNIA REGION',
 'EAST GEORGIA REGIONAL MED CTR',
 'NEW LONDON HOSPITAL',
 'RAPID RADIOLOGY, INC.',
 'DIALCARE, LLC',
 'OZARKS MEDICAL CENTER',
 'TELESPECIALISTS, LLC',
 'LOGANSPORT MEMORIAL HOSPITAL',
 'LYRA HEALTH',
 'ARMED FORCES RETIREMENT HOME',
 'ST JOHN HLTH SYS CNTRL VERIFICATION OFC.',
 'OREGON STATE HOSPITAL',
 'SURGERY CENTER OF TEXAS',
 'CARMEL AMBULATORY SURGERY CENTER',
 'ESTES PARK MEDICAL CENTER',
 'CONIFER VALUE BASED CARE',
 'MAT-SU REGIONAL MEDICAL CENTER',
 'RIVERPARK AMBULATORY SURGERY CENTER',
 'NAMM CALIFORNIA',
 'PREMIER ER PLUS,LLC',
 'ST. DOMINIC-JACKSON MEMORIAL HOSPITAL',
 'BSA HEALTH SYSTEMS',
 'LOC

In [59]:
customer_list

['RIVER REGION HEALTH SYSTEM',
 'ORO VALLEY HOSPITAL',
 'UNIVERSITY OF MARYLAND CAPITAL REGION HEALTH',
 'CHILDRENS HOSPITAL CENTRAL CALIFORNIA',
 "UNITYPOINT ST. LUKE'S HOSPITAL - CEDAR RAPIDS",
 'MONROE CLINIC',
 'HEALTHCARE AUTHORITY FOR MEDICAL WEST',
 'NORTH TAMPA BEHAVIORAL HEALTH',
 'MISSISSIPPI VALLEY SURGERY CENTER',
 'PROVIDENCE HEALTH AND SERVICES CALIFORNIA REGION',
 'EAST GEORGIA REGIONAL MED CTR',
 'NEW LONDON HOSPITAL',
 'RAPID RADIOLOGY, INC.',
 'DIALCARE, LLC',
 'OZARKS MEDICAL CENTER',
 'TELESPECIALISTS, LLC',
 'LOGANSPORT MEMORIAL HOSPITAL',
 'LYRA HEALTH',
 'ARMED FORCES RETIREMENT HOME',
 'ST JOHN HLTH SYS CNTRL VERIFICATION OFC.',
 'OREGON STATE HOSPITAL',
 'SURGERY CENTER OF TEXAS',
 'CARMEL AMBULATORY SURGERY CENTER',
 'ESTES PARK MEDICAL CENTER',
 'CONIFER VALUE BASED CARE',
 'MAT-SU REGIONAL MEDICAL CENTER',
 'RIVERPARK AMBULATORY SURGERY CENTER',
 'NAMM CALIFORNIA',
 'PREMIER ER PLUS,LLC',
 'ST. DOMINIC-JACKSON MEMORIAL HOSPITAL',
 'BSA HEALTH SYSTEMS',
 'LOC

In [60]:
for name in customer_list:
    v = False
    if 'VETERAN' in name:
        print(name)
        v = True
    elif 'VAMC' in name:
        print(name)
        v = True
    elif ' VA ' in name:
        print(name)
        v = True
    if v==True:
        new_dict = {
                'CUSTOMER_NAME': customer,
                'OFFICE_ADDRESS_LINE_1': reference,
                'SCORE': score
            }
        dict_list.append(new_dict)

PHILADELPHIA VA MEDICAL CENTER
JAMES E. VAN ZANDT VA MEDICAL CENTER
EDWARD HINES JR. VA  HOSPITAL
VA GULF COAST VETERANS HLTH CARE SYS
CENTRAL ALABAMA VETERANS HEALTHCARE SYSTEM
FAYETTEVILLE VETERANS ADMINISTRATION MEDICAL CENTER
ATLANTA VAMC
ORLANDO VA MEDICAL CENTER
JESSE BROWN VA MEDICAL CENTER
PITTSBURGH VA HEALTHCARE SYSTEM
HARRY S TRUMAN MEMORIAL VETERANS' HOSPITAL
WM. S. MIDDLETON MEMORIAL VA HOSPITAL
PORTLAND VA MEDICAL CENTER
SOUTH TEXAS VETERANS HEALTH CARE SYSTEM
CARL T. HAYDEN VA MED CTR (05D)
CLEMENT J. ZABLOCKI VA MEDICAL CENTER
VETERANS AFFAIRS MEDICAL CENTER
ROUDEBUSH VA MEDICAL CENTER
MINNEAPOLIS VA MEDICAL CENTER
LOUIS STOKES CLEVELAND VA MEDICAL CENTER
SAN FRANCISCO VA HEALTH CARE SYSTEM
CENTRAL TEXAS VETERANS HEALTHCARE SYSTEM
BOISE VETERANS AFFAIRS MEDICAL CENTER
BECKLEY VA MEDICAL CENTER


In [61]:
dict_list

[{'CUSTOMER_NAME': 'RIVER REGION HEALTH SYSTEM',
  'OFFICE_ADDRESS_LINE_1': 'RIVER REGION HEALTH SYSTEM',
  'SCORE': 100},
 {'CUSTOMER_NAME': 'ORO VALLEY HOSPITAL',
  'OFFICE_ADDRESS_LINE_1': 'ORO VALLEY HOSPITAL',
  'SCORE': 100},
 {'CUSTOMER_NAME': 'CHILDRENS HOSPITAL CENTRAL CALIFORNIA',
  'OFFICE_ADDRESS_LINE_1': "CHILDREN'S HOSPITAL CENTRAL CALIFORNIA",
  'SCORE': 99},
 {'CUSTOMER_NAME': 'MONROE CLINIC',
  'OFFICE_ADDRESS_LINE_1': 'MONROE CLINIC',
  'SCORE': 100},
 {'CUSTOMER_NAME': 'NEW LONDON HOSPITAL',
  'OFFICE_ADDRESS_LINE_1': 'NEW LONDON HOSPITAL',
  'SCORE': 100},
 {'CUSTOMER_NAME': 'OZARKS MEDICAL CENTER',
  'OFFICE_ADDRESS_LINE_1': 'OZARK MEDICAL CENTER',
  'SCORE': 98},
 {'CUSTOMER_NAME': 'LOGANSPORT MEMORIAL HOSPITAL',
  'OFFICE_ADDRESS_LINE_1': 'LOGANSPORT MEMORIAL HOSPITAL',
  'SCORE': 100},
 {'CUSTOMER_NAME': 'ARMED FORCES RETIREMENT HOME',
  'OFFICE_ADDRESS_LINE_1': 'ARMED FORCES RETIREMENT HOME',
  'SCORE': 100},
 {'CUSTOMER_NAME': 'OREGON STATE HOSPITAL',
  'OFFIC

In [78]:
len(unclass)

10472

In [98]:
manual = pd.read_excel('../../Data/Credentialling/may_manual.xlsx')
manual = manual.fillna('None')

In [108]:
DICT_LIST = []
for row in manual.itertuples():
    if row.city=='None' and row.address!='None':
        
        print(row.address)
        add_list = row.address.split(', ')
        address = add_list[0]
        print(address)
        try:
            city = add_list[1]
            state = add_list[2].split(' ')[0]
            zipcode = add_list[2].split(' ')[1]
        except:
            city = 'None'
            state = 'None'
            zipcode = 'None'
    else:
        address = row.address
        city = row.city
        state = row.state
        zipcode = row.zipcode
    new_dict = {
        'Address': address,
        'City': city,
        'State': state,
        'Zipcode': zipcode,
        'CUSTOMER_KEY':row.CUSTOMER_KEY
    }
    DICT_LIST.append(new_dict)

 2601 E Roosevelt St, Phoenix, AZ 85008
 2601 E Roosevelt St
 2893 Sturgis Rd, Rock Hill, SC 29730
 2893 Sturgis Rd
1 Riverway Suite 700, Houston, TX 77056
1 Riverway Suite 700
1 Tampa General Cir, Tampa, FL 33606
1 Tampa General Cir
100 Hospital Rd, Prince Frederick, MD 20678
100 Hospital Rd, Prince Frederick, MD 20678
100 Medical Center Dr, Princeton, KY 42445
100 Medical Center Dr
1000 36th St Vero Beach, FL 32960 United States
1000 36th St Vero Beach
1000 Medical Center Dr, Hardeeville, SC 29927
1000 Medical Center Dr
10000 West Colonial Drive Ocoee, FL 34761 
10000 West Colonial Drive Ocoee
101 Dudley St, Providence, RI 02905
101 Dudley St
101 E 9th St, Pana, IL 62557
101 E 9th St
10101 Park Rowe Cir # 500, Baton Rouge, LA 70810
10101 Park Rowe Cir # 500
10105 Park Rowe Cir, Baton Rouge, LA 70810
10105 Park Rowe Cir
1011 N Galloway Ave, Mesquite, TX 75149
1011 N Galloway Ave
10333 El Camino Real, Atascadero, CA 93422
10333 El Camino Real
105 Highway 80 East, Demopolis, AL 36732
10

In [106]:
z = '100 Hospital Rd, Prince Frederick, MD 20678'
zz = z.split(', ')
zz[2].split(' ')[0]

'MD'

In [109]:
DICT_LIST

[{'Address': '\xa02601 E Roosevelt St',
  'City': 'Phoenix',
  'State': 'AZ',
  'Zipcode': '85008',
  'CUSTOMER_KEY': 131103},
 {'Address': '\xa02893 Sturgis Rd',
  'City': 'Rock Hill',
  'State': 'SC',
  'Zipcode': '29730',
  'CUSTOMER_KEY': 60149},
 {'Address': '\xa08101 W Sam Houston Pkwy S\xa0 # Sui\xa0',
  'City': 'Houston,\xa0 TX\xa0 77072\xa0',
  'State': 'None',
  'Zipcode': 'None',
  'CUSTOMER_KEY': 59997},
 {'Address': '1 Riverway Suite 700',
  'City': 'Houston',
  'State': 'TX',
  'Zipcode': '77056',
  'CUSTOMER_KEY': 60526},
 {'Address': '1 Tampa General Cir',
  'City': 'Tampa',
  'State': 'FL',
  'Zipcode': '33606',
  'CUSTOMER_KEY': 131621},
 {'Address': '100 Hospital Rd,\xa0Prince Frederick,\xa0MD\xa020678',
  'City': 'None',
  'State': 'None',
  'Zipcode': 'None',
  'CUSTOMER_KEY': 57760},
 {'Address': '100 Medical Center Dr',
  'City': 'Princeton',
  'State': 'KY',
  'Zipcode': '42445',
  'CUSTOMER_KEY': 60754},
 {'Address': '1000 36th St Vero Beach',
  'City': 'None',

In [19]:
ref = pd.read_csv('../../Data/Credentialling/Reference_Match.csv')

In [49]:
matched_orders = pd.merge(ref, customer_orders, on = 'CUSTOMER_NAME')

In [48]:
customer_orders['CUSTOMER_NAME'] = [x.upper() for x in list(customer_orders['CUSTOMER_NAME'])]

In [50]:
matched_orders = matched_orders.fillna('None')

In [51]:
attempt = matched_orders[matched_orders['MAILING_ADDRESS_1']!='None']

In [52]:
attempt[['CUSTOMER_NAME', 'NAME', 'MAILING_ADDRESS_1', 'MAILING_ADDRESS_2', 'MAILING_CITY',
       'MAILING_STATE', 'MAIING_ZIP', 'LOCATION_ADDRESS_1',
       'LOCATION_ADDRESS_2', 'LOCATION_CITY', 'LOCATION_STATE', 'LOCATION_ZIP', 'FULL_DT', 'ME', 'PARTY_ID',
       'ORDER_PRODUCT_ID', 'CUSTOMER_KEY', 'CUSTOMER_NBR']].to_excel(('../../Data/Credentialling/Customers_pre-2020.xlsx'))

In [53]:
len(attempt)

707507

In [54]:
len(customer_orders)

2430136

In [57]:
len(attempt[attempt['ORDER_PRODUCT_ID'].isin([4915513,4915521,4915502])])

506810

In [60]:
HOSP_CUST.groupby('CUSTOMER_CATEGORY_DESC').count()

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE
CUSTOMER_CATEGORY_DESC,,,,,,
Ambulatory Care,7853,7853,7853,7853,7853,7853
American Board,1,1,1,1,1,1
Consulting,116,116,116,116,116,116
County Medical Society,129,129,129,129,129,129
Credentialling Board,283,283,283,283,283,283
Credit Verification Org,284,284,284,284,284,284
Government,184,184,184,184,184,184
Group,1736,1736,1736,1736,1736,1736
Health Related,2051,2051,2051,2051,2051,2051


In [50]:
HOSP_CUST

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,1,000000105762,35HP10545,Nassau County Medical Center,Eprofile Webstore Cusomer,Eprofile,Hospital
1,2,000000545697,None,Abdul Haji,Non-Physician Individual,Nonphy,None
2,3,000000552782,None,Mary Tan,Non-Physician Individual,Nonphy,None
3,4,000000647381,None,Osman Kozak,Physician (On Aims),Phys,None
4,5,000000648448,None,Rana Kayal,Non-Physician Individual,Nonphy,None
...,...,...,...,...,...,...,...
137925,138040,002604831507,None,Michael P Manning,Physician (On Aims),Phys,None
137926,138041,003006060331,None,Katharine Pico,Physician (On Aims),Phys,None
137927,138042,004116150455,None,George Salama,Us Physician Resident/Fellow,Resident,None
137928,138043,001103170628,None,Vishal K Goswami,Us Physician Resident/Fellow,Resident,None


In [51]:
ref = pd.read_csv("../../Data/Credentialling/Reference_Match.csv")

In [52]:
ref

,CUSTOMER_NAME,NAME,SCORE,NPI,Taxonomies,OTHER_NAME,MAILING_ADDRESS_1,MAILING_ADDRESS_2,MAILING_CITY,MAILING_STATE,MAIING_ZIP,MAILING_PHONE,LOCATION_ADDRESS_1,LOCATION_ADDRESS_2,LOCATION_CITY,LOCATION_STATE,LOCATION_ZIP,LOCATION_PHONE,LAST_UPDATE_DATE
0,633D MEDICAL GROUP/SGHC,633D MEDICAL GROUP,88.0,1.609860e+09,"['261QM1100X', '261QM1101X', '2865M2000X', '33...",1ST MEDICAL GROUP,77 NEALY AVE,None,LANGLEY AFB,VA,236652040,7577646807,77 NEALY AVE,None,LANGLEY AFB,VA,236652040.0,7577646807,3/3/2014
1,A.O. FOX MEMORIAL HOSPITAL,A.O FOX MEMORIAL HOSPITAL,98.0,1.104029e+09,['282N00000X'],None,2 ELLIOTTS ALY,None,VALLEY COTTAGE,NY,109892306,6462395675,1 NORTON AVE,None,ONEONTA,NY,138202629.0,6074315243,7/8/2007
2,ABBEVILLE COUNTY MEMORIAL HOSPITAL,ABBEVILLE COUNTY MEMORIAL HOSPITAL,100.0,1.063449e+09,"['282NC0060X', '3336C0003X']",ABBEVILLE AREA MEDICAL CENTER,PO BOX 887,None,ABBEVILLE,SC,296200887,8643363249,420 THOMSON CIR,None,ABBEVILLE,SC,29620.0,8643665011,8/23/2018
3,ABBEVILLE GENERAL HOSPITAL,ABBEVILLE GENERAL HOSPITAL,100.0,1.255300e+09,['282NR1301X'],None,118 N HOSPITAL DR,None,ABBEVILLE,LA,705104039,3378935466,118 N HOSPITAL DR,None,ABBEVILLE,LA,705104039.0,3378935466,4/20/2008
4,ABILENE BEHAVIORAL HEALTH,"ABILENE BEHAVIORAL HEALTH, LLC",91.0,1.740332e+09,['283Q00000X'],"ACADIA ABILENE, LLC",830 CRESENT CENTRE DRIVE,SUITE 610,FRANKLIN,TN,37067,6158616000,4225 WOODS PLACE,None,ABILENE,TX,79602.0,3256986600,7/30/2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216,SAGE REHABILITATION HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4217,UNION COUNTY HOSPITAL DISTRICT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4218,MARTIN GENERAL HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4219,DVA MEDICAL CENTER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
HOSP_CUST['CUSTOMER_NAME']=[x.upper() for x in HOSP_CUST.CUSTOMER_NAME]
pd.merge(ref, HOSP_CUST, on='CUSTOMER_NAME')[['CUSTOMER_KEY','NPI']].dropna().to_csv("../../Data/Credentialling/Reference_Basic.csv")

In [14]:
ppd.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [15]:
ppd.FIPS_COUNTY

0          117.0
1          183.0
2          119.0
3           13.0
4           71.0
           ...  
1283455    135.0
1283456      3.0
1283457     81.0
1283458     81.0
1283459    113.0
Name: FIPS_COUNTY, Length: 1283460, dtype: float64